## Código para identificar tiendas Cornershop por código postal

Con base en los códigos postales de SEPOMEX, se identifican las tiendas Cornershop que se encuentran en cada uno de los códigos postales.

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time
import requests
import random
import json

In [8]:
#Directorio de trabajo
try:
    os.chdir('C:/Users/claudio.pacheco/Documents/GitHub/pacic/')
except:
    os.chdir("D:/")

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


#Parámetros
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
options.add_argument("-inprivate")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [14]:
def data_walmart(zip_code, busqueda):
    busqueda = busqueda.replace(" ", "+")
    
    #Cambia ubicación para obtener precios
    url = "https://super.walmart.com.mx/"
    url_busqueda_template = "https://super.walmart.com.mx/search?q=" + busqueda + "&page={}"
    
    driver = webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(5)
    
    try:
        elemento = driver.find_element(By.NAME,"select-store-button")
    except NoSuchElementException:
        elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')
    elemento.click()

    #Ingresa código postal
    time.sleep(4)
    cp = driver.find_element(By.CSS_SELECTOR, "input[data-automation-id='store-zip-code']")
    cp.clear()
    cp.send_keys(zip_code)
    time.sleep(4)
    
    # Attempt to retrieve store details
    try:
        tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
        address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
        radio_buttons = driver.find_element(By.NAME, 'pickup-store')
    except NoSuchElementException:
        # If any of the above elements are not found, return an empty DataFrame.
        print(f"No existe Walmart cercano al código postal {zip_code}.")
        return pd.DataFrame()

    #Aceptar elección
    elegir = driver.find_element(By.CSS_SELECTOR, "button[data-automation-id='save-label']")
    elegir.click()
    time.sleep(4)

    all_data = []
    for page_number in range(1, 2):
        url_busqueda = url_busqueda_template.format(page_number)
        driver.get(url_busqueda)
        time.sleep(4)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        results = soup.find_all('div', {'data-stack-index': '0'})
        
        span_tags = []
        product_titles = []
        
        # Loop para obtener precios y productos
        for result in results:
            span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
            product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

        precios = []
        for span in span_tags:
            text = span.text
            if 'precio actual' in text:
                precios.append(text.split("precio actual")[1].strip())

        fecha = time.strftime("%d/%m/%y")
        hora = time.strftime("%H:%M:%S")
        fecha_hora = hora + " " + fecha

        dfx = pd.DataFrame({
            'producto': product_titles,
            'precio': precios,
            'tienda': tienda,
            'direccion': address,
            'busqueda': busqueda,
            'fecha_consulta': fecha_hora
        })
        
        all_data.append(dfx)
        time.sleep(5)

    df = pd.concat(all_data, ignore_index=True)
    
    # Limpiar precios
    df['precio'] = df['precio'].str.replace(r'[^0-9.]', '', regex=True)  # Dejar solo números utilizando expresiones regulares
    df['precio'] = df['precio'].str.replace(',', '', regex=True)  # Eliminar comas
    df['precio'] = pd.to_numeric(df['precio'])
    df['producto'] = df.producto.str.lower()

    return df




In [15]:
walmart=data_walmart('06720','bicicleta infantil')
walmart["tienda"]="Walmart Portal Centro"
walmart

,producto,precio,tienda,direccion,busqueda,fecha_consulta
0,botella infantil mainstays astronauta 400 ml,35.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
1,botella infantil mainstays dinosaurio 420 ml,35.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
2,pan infantil paquete con 10 piezas,45.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
3,trampolín infantil plum diseño océano con sonido,1999.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
4,bicicleta infantil huffy wonder woman rodada 1...,1619.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
5,tempra infantil 160 mg 30 tabletas masticables,189.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
6,bicicleta infantil huffy tantrum rodada 20 ana...,4229.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
7,bicicleta infantil huffy rodada 16 mod 51492m ...,3119.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
8,silla infantil varios colores 1 pieza,349.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23
9,redoxon inf redoxon 100 mg 100 tabletas mastic...,124.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23


### Coppel

In [44]:
url="https://www.coppel.com/bicicletas-infantiles?fromPage=catalogEntryList&beginIndex=0&pageSize=72&pageView=grid"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [45]:
r=requests.get(url, headers=headers)
r.status_code

200

In [46]:
soup=BeautifulSoup(r.content, "html.parser")

In [47]:
results=soup.find_all("div", {"class":"product_info"})

In [48]:
productos=[]
precios=[]

for result in results:
    product_name_element = result.find("div", {"class":"product_name"})
    if product_name_element:
        productos.append(product_name_element.text.strip())
    else:
        productos.append(None)

    price_element = result.find("span", {"class":"price"})
    if price_element:
        precios.append(price_element.text.strip())
    else:
        precios.append(None)


In [49]:
coppel=pd.DataFrame({"producto":productos, "precio":precios, "tienda":"Coppel"})
#Eliminar signos de pesos
coppel["precio"]=coppel["precio"].str.replace("$","", regex=True)
#Eliminar comas
coppel["precio"]=coppel["precio"].str.replace(",","", regex=True)
#Convertir a float
coppel["precio"]=coppel["precio"].astype(float)
#Eliminar "\n" y palabra "Oferta"
coppel["producto"]=coppel["producto"].str.replace("\n","")
coppel["producto"]=coppel["producto"].str.replace("Oferta","")


#Eliminar registros que contengan "Exclusivo"
coppel=coppel[~coppel["producto"].str.contains("Exclusivo")]
#Incorporar busqueda
coppel["busqueda"]="bicicleta+infantil"
#Incorporar fecha de consulta
coppel["fecha_consulta"]=time.strftime("%d/%m/%y") + " " + time.strftime("%H:%M:%S")
coppel

,producto,precio,tienda,busqueda,fecha_consulta
16,"Bicicleta Infantil Pro Thunder 16"" AzulPrecio...",2699.0,Coppel,bicicleta+infantil,07/09/23 19:01:46
48,"Bicicleta Mongoose 20"" PlateadaPrecio contado:",4999.0,Coppel,bicicleta+infantil,07/09/23 19:01:46
51,"Bicicleta Infantil Huffy Summerland 20"" AzulPr...",3199.0,Coppel,bicicleta+infantil,07/09/23 19:01:46
52,"Bicicleta Infantil Yamaha Motobike 16"" AzulPre...",NaN,Coppel,bicicleta+infantil,07/09/23 19:01:46
55,Bicicleta Freestyle Rising Fit Ninja 20'' Negr...,3899.0,Coppel,bicicleta+infantil,07/09/23 19:01:46
65,"Bicicleta Swift Flower 16"" NegraPrecio contado:",2399.0,Coppel,bicicleta+infantil,07/09/23 19:01:46
71,"Bicicleta Huffy Summertime 16""Precio contado:",1999.0,Coppel,bicicleta+infantil,07/09/23 19:01:46


# Liverpool

In [22]:
url="https://www.liverpool.com.mx/tienda?s=bicicleta+infantil"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}


In [23]:
r=requests.get(url, headers=headers)
r.status_code

200

In [24]:
soup=BeautifulSoup(r.content, "html.parser")

In [25]:
script_tag = soup.find('script', attrs={"id": "__NEXT_DATA__"})

In [26]:
if script_tag:
    json_content = script_tag.string
    json_data = json.loads(json_content)

In [27]:
productos=[]
precios=[]
for i in json_data["query"]["data"]["mainContent"]["records"]:
    productos.append(i["allMeta"]["title"])
    precios.append(i["allMeta"]["maximumPromoPrice"])
    

In [28]:
liverpool=pd.DataFrame({"producto":productos, "precio":precios, "tienda":"Liverpool"})
#Agregar busqueda
liverpool["busqueda"]="bicicleta+infantil"
#Agregar fecha de consulta
liverpool["fecha_consulta"]=time.strftime("%d/%m/%y") + " " + time.strftime("%H:%M:%S")
liverpool

,producto,precio,tienda,busqueda,fecha_consulta
0,Bicicleta infantil Veloci Bikes Veloci Kids,2399.40,Liverpool,bicicleta+infantil,07/09/23 18:59:35
1,Bicicleta infantil Mercurio rodada 16 Troya,2699.10,Liverpool,bicicleta+infantil,07/09/23 18:59:35
2,Bicicleta infantil Veloci Bikes rodada 12 Patr...,1379.40,Liverpool,bicicleta+infantil,07/09/23 18:59:35
3,Bicicleta infantil Turbo rodada 20 \n015874 Be...,3899.40,Liverpool,bicicleta+infantil,07/09/23 18:59:35
4,Bicicleta Veloci rodada 12 Paw Patrol Big infa...,1858.29,Liverpool,bicicleta+infantil,07/09/23 18:59:35
5,Bicicleta Turbo rodada 16 015869 Turbo Bucaner...,2299.50,Liverpool,bicicleta+infantil,07/09/23 18:59:35
6,Bicicleta infantil The Baby Shop rodada 12 Uni...,1899.90,Liverpool,bicicleta+infantil,07/09/23 18:59:35
7,Bicicleta infantil The Baby Shop rodada 12 Uni...,1899.90,Liverpool,bicicleta+infantil,07/09/23 18:59:35
8,Bicicleta infantil Mercurio rodada 16 Spyro pa...,2519.10,Liverpool,bicicleta+infantil,07/09/23 18:59:35
9,Bicicleta infantil Little Monkey rodada 16 unisex,3899.00,Liverpool,bicicleta+infantil,07/09/23 18:59:35


## Palacio de Hierro

In [29]:
url="https://www.elpalaciodehierro.com/buscar?q=bicicleta-infantil&start=0&sz=100"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [30]:
r=requests.get(url, headers=headers)
r.status_code

200

In [31]:
soup=BeautifulSoup(r.content, "html.parser")

In [32]:
results=soup.find_all("div", {"class":"b-product"})

In [33]:
# Iterate over each div and extract the desired information
products = []
for div in results:
    data_analytics = div.get('data-analytics')
    if data_analytics:
        product_data = json.loads(data_analytics)['product']
        product_info = {
            'producto': product_data['name'],
            'precio': product_data['price'],
            'marca': product_data['brand'],
        }
        products.append(product_info)

In [34]:
#Crear dataframe
palacio=pd.DataFrame(products)
palacio['tienda'] = 'Palacio de Hierro'
#Agregar busqueda
palacio["busqueda"]="bicicleta+infantil"
#Agregar fecha de consulta
palacio["fecha_consulta"]=time.strftime("%d/%m/%y") + " " + time.strftime("%H:%M:%S")
palacio

,producto,precio,marca,tienda,busqueda,fecha_consulta
0,"Bicicleta de Montaña Sniper R20"" 21V",6209.0,Benotto,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
1,Bicicleta para bebé de equilibrio sin pedales,4699.0,Hape,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
2,Bicicleta Racing 2.0 Niño,4740.0,Turbo,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
3,Bicicleta Princess Niña,3230.0,Huffy,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
4,Bicicleta Summerland Niña,3699.0,Huffy,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
...,...,...,...,...,...,...
61,Set 4 Llantas 72 mm con Luz Led Verde,479.0,Dmsport,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
62,Set 4 Llantas 72 mm con Luz Led Rosa,479.0,Dmsport,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
63,Set 4 Llantas 64 mm con Luz Led Rosa,449.0,Dmsport,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50
64,Set 4 Llantas 64 mm con Luz Led Verde,449.0,Dmsport,Palacio de Hierro,bicicleta+infantil,07/09/23 18:59:50


## Elektra


In [35]:
url="https://www.elektra.mx/bicicleta%20infantil?_q=bicicleta%20infantil&map=ft&page=1"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [36]:
r=requests.get(url, headers=headers)

In [37]:
soup = BeautifulSoup(r.text, "html.parser")

In [38]:
templates = soup.find_all('template', attrs={"data-type": "json"})

In [39]:
productos = []
for template in templates:
    for script in template.find_all('script'):
        json_content = script.string
        data = json.loads(json_content)

In [40]:
productos = [product['productName'] for product in data.values() if 'productName' in product]
marca=[product['brand'] for product in data.values() if 'brand' in product]
precios=[product['highPrice'] for product in data.values() if 'highPrice' in product]
#Dejar solo los precios de 1,3,5,7,9 y así sucesivamente
precios=precios[::2]

In [41]:
elektra=pd.DataFrame({"producto":productos, "precio":precios, "marca":marca, "tienda":"Elektra"})
#Agregar busqueda
elektra["busqueda"]="bicicleta+infantil"
#Agregar fecha de consulta
elektra["fecha_consulta"]=time.strftime("%d/%m/%y") + " " + time.strftime("%H:%M:%S")
elektra

,producto,precio,marca,tienda,busqueda,fecha_consulta
0,Bicicleta Infantil Benotto Chamoy R20 1V Magen...,4199,BENOTTO,Elektra,bicicleta+infantil,07/09/23 19:00:12
1,Bicicleta Infantil Mercurio Mtb Troya R16 1V A...,2599,MERCURIO,Elektra,bicicleta+infantil,07/09/23 19:00:12
2,Bicicleta Infantil Benotto Agressor R16 1V Azul,3799,BENOTTO,Elektra,bicicleta+infantil,07/09/23 19:00:12
3,Bicicleta Infantil Roja Niño R16 Llantas Entre...,2499,LITTLE MONKEY,Elektra,bicicleta+infantil,07/09/23 19:00:12
4,Bicicleta Infantil Distroller Chamoy R16 1V Ro...,3199,DISTROLLER,Elektra,bicicleta+infantil,07/09/23 19:00:12
5,Bicicleta Infantil Mercurio Sweet Girl R20 1V ...,2699,MERCURIO,Elektra,bicicleta+infantil,07/09/23 19:00:12
6,Bicicleta Infantil Benotto Agressor R20 1V Neg...,3799,BENOTTO,Elektra,bicicleta+infantil,07/09/23 19:00:12
7,Bicicleta Infantil Benotto Flower Power R20 1V...,4299,BENOTTO,Elektra,bicicleta+infantil,07/09/23 19:00:12
8,Bicicleta Infantil Veloci Amore R16 1V Rosa,2499,VELOCI,Elektra,bicicleta+infantil,07/09/23 19:00:12
9,Bicicleta Infantil Supermex Ukrania R20 1V Negro,3199,SUPERMEX,Elektra,bicicleta+infantil,07/09/23 19:00:12


## Pegar dataframes



In [50]:
df_final=pd.concat([walmart,coppel,liverpool,palacio,elektra],axis=0)

In [51]:
df_final

,producto,precio,tienda,direccion,busqueda,fecha_consulta,marca
0,botella infantil mainstays astronauta 400 ml,35.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
1,botella infantil mainstays dinosaurio 420 ml,35.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
2,pan infantil paquete con 10 piezas,45.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
3,trampolín infantil plum diseño océano con sonido,1999.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
4,bicicleta infantil huffy wonder woman rodada 1...,1619.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
...,...,...,...,...,...,...,...
55,Bicicleta Mercurio Spyro 16 Color Amarilla,3741.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,MERCURIO
56,Bicicleta de Montaña Ghost Claw Rodada 29 Negra,3073.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,VELOCI
57,Bicicleta Black Panther Bravia Rodada 20 Equip...,3045.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,BLACK PANTHER
58,Bicicleta Black Panther Bravia Rodada 20 Equip...,3045.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,BLACK PANTHER


In [53]:
#Strings to lower
df_final["producto"]=df_final["producto"].str.lower()
df_final["marca"]=df_final["marca"].str.lower()
df_final["busqueda"]=df_final["busqueda"].str.lower()
df_final["marca"]=df_final["marca"].str.lower()


In [54]:
#Dejar solo registros que contengan la palabra bicicleta en producto
df_final=df_final[df_final["producto"].str.contains("bicicleta")]
df_final

,producto,precio,tienda,direccion,busqueda,fecha_consulta,marca
4,bicicleta infantil huffy wonder woman rodada 1...,1619.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
6,bicicleta infantil huffy tantrum rodada 20 ana...,4229.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
7,bicicleta infantil huffy rodada 16 mod 51492m ...,3119.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
10,bicicleta huffy tantrum rodada 26,5359.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
12,bicicleta ozark trail infantil sin suspensión ...,2199.0,Walmart Portal Centro,"Av. Lorenzo Boturini No. 258, Colonia Tránsit...",bicicleta+infantil,18:58:40 07/09/23,NaN
...,...,...,...,...,...,...,...
55,bicicleta mercurio spyro 16 color amarilla,3741.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,mercurio
56,bicicleta de montaña ghost claw rodada 29 negra,3073.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,veloci
57,bicicleta black panther bravia rodada 20 equip...,3045.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,black panther
58,bicicleta black panther bravia rodada 20 equip...,3045.0,Elektra,NaN,bicicleta+infantil,07/09/23 19:00:12,black panther


In [57]:
#Guardar como csv
df_final.to_csv("bicicleta_infantil.csv", index=False, encoding="latin1")

In [56]:
import os
os.getcwd()


'C:\\Users\\claudio.pacheco\\Documents\\GitHub\\pacic'

In [59]:
#Imprimir número de tiendas únicas y largo de la base
print("Número de tiendas únicas: ", df_final["tienda"].nunique())
#Tiendas únicas
print("Tiendas únicas: ", df_final["tienda"].unique())
print("Número de registros: ", len(df_final))

Número de tiendas únicas:  5
Tiendas únicas:  ['Walmart Portal Centro' 'Coppel' 'Liverpool' 'Palacio de Hierro'
 'Elektra']
Número de registros:  185
